In [9]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Deliverable2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [11]:
# Extract the dataset and load data into Spark Dataframe
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [13]:
# Filter reviews that are more likely to be helpful
filtered_vine_df_1 = vine_df.filter('total_votes>=20')
filtered_vine_df_1.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [14]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is 
# equal to or greater than 50%
filtered_vine_df_2 = filtered_vine_df_1.filter('(helpful_votes/total_votes)>=0.5')
filtered_vine_df_2.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [15]:
# Retrieves all the rows where a review was written as part of the Vine program (paid)
from pyspark.sql.functions import col
filtered_vine_df = filtered_vine_df_2.filter(col("vine") == 'Y')
filtered_vine_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R2XRYNV2SY3ZKL|          5|           53|         56|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R2QVTDYYLTP8SL|          5|           21|         24|   Y|     

In [16]:
# Retrieves all the rows where the review was not part of the Vine program (unpaid)
filtered_not_vine_df = filtered_vine_df_2.filter(col("vine") == 'N')
filtered_not_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [17]:
# Calculate the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the paid reviews
total_review_vine = filtered_vine_df.count()
total_5_star_review_vine = filtered_vine_df.filter("star_rating==5").count()
percentage_5_star_review_vine = total_5_star_review_vine/total_review_vine*100

In [18]:
# Create a summary table for the paid reviews
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
data = [(total_review_vine,total_5_star_review_vine, percentage_5_star_review_vine)]

schema = StructType([ \
    StructField("total number of reviews",IntegerType(),True), \
    StructField("number of 5-star reviews",IntegerType(),True), \
    StructField("percentage of 5-star reviews",FloatType(),True), \
  ])
 
paid_summary = spark.createDataFrame(data=data,schema=schema)
paid_summary.show()

+-----------------------+------------------------+----------------------------+
|total number of reviews|number of 5-star reviews|percentage of 5-star reviews|
+-----------------------+------------------------+----------------------------+
|                     22|                      13|                   59.090908|
+-----------------------+------------------------+----------------------------+



In [19]:
# Calculate the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the unpaid reviews
total_review_not_vine = filtered_not_vine_df.count()
total_5_star_review_not_vine = filtered_not_vine_df.filter("star_rating==5").count()
percentage_5_star_review_not_vine = total_5_star_review_not_vine/total_review_not_vine*100

In [20]:
# Create a summary table for the unpaid reviews
data1 = [(total_review_not_vine,total_5_star_review_not_vine, percentage_5_star_review_not_vine)]

schema1 = StructType([ \
    StructField("total number of reviews",IntegerType(),True), \
    StructField("number of 5-star reviews",IntegerType(),True), \
    StructField("percentage of 5-star reviews",FloatType(),True), \
  ])
 
unpaid_summary = spark.createDataFrame(data=data1,schema=schema1)
unpaid_summary.show()

+-----------------------+------------------------+----------------------------+
|total number of reviews|number of 5-star reviews|percentage of 5-star reviews|
+-----------------------+------------------------+----------------------------+
|                  26987|                   14475|                   53.636936|
+-----------------------+------------------------+----------------------------+



In [22]:
# Create the product DataFrame
products_df = df.select(["review_id", "product_title", "product_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
products_df.show(10)

+--------------+--------------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|Minnetonka Men's ...|B000XB31C0|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|Teva Men's Pajaro...|B00CFYZH5W|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|Anne Klein Perfec...|B00S8JNN3Q|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|adidas Men's 10K ...|B00XFBPOQG|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|OverBling Sneaker...|B00SW64Y9W|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|MESSI 15.3 FG/AG ...|B011F9E6LI|          5|            1|          1|   N|                Y|
|R12ENYLFG

In [24]:
# filter the products_df for rows with hepful ratings
from pyspark.sql.functions import col
filtered_products_df = products_df.filter('total_votes>=20').filter('(helpful_votes/total_votes)>=0.5')
filtered_products_df.show(5)

+--------------+--------------------+----------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|Travelon Converti...|B00TQ6L9TK|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|Hoxis Tassel Faux...|B00UJ6B33I|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|Adidas Yeezy Boos...|B010KK08LU|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|Gucci GG1008 Eyeg...|B009K1WFAI|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|KENSBUY Men's Lig...|B0117P7DV0|          2|           19|         22|   N|                Y|
+--------------+--------------------+----------+-----------+-------------+-----------+----+-----------------+
only showi

In [41]:
# create a products dataframe for the paid group and retrieve the average ratingof each product
from pyspark.sql.functions import avg
vine_products_df = filtered_products_df.filter(col("vine") == 'Y').groupBy("product_id").agg(avg("star_rating")).withColumnRenamed("avg(star_rating)", "Vine avg rating")
vine_products_df.show()

+----------+---------------+
|product_id|Vine avg rating|
+----------+---------------+
|B00NHUW1NE|            5.0|
|B005P1YV90|            5.0|
|B00DOEA73O|            4.0|
|B00KQ2VQDC|            4.0|
|B00NHUVF18|            3.0|
|B00D8HXHYI|            5.0|
|B00DOE9QFY|            5.0|
|B00M15DG80|            4.0|
|B00NHUV8SI|            4.0|
|B00NHUVTPA|            5.0|
|B00NHUVFGS|            4.5|
|B00NHUVN10|            2.0|
|B00NHUW1UW|            4.0|
|B00NHUVTSM|            5.0|
|B00M32FLL6|            5.0|
|B00KQ57KWK|            5.0|
|B00LNN1EZ8|            5.0|
|B00R9LA030|            5.0|
|B00D8J693W|            5.0|
|B00IF3BG6G|            2.0|
+----------+---------------+



In [35]:
# convert the product ids in the paid group to a list
vine_products = [data[0] for data in vine_products_df.select('product_id').collect()]
print(vine_products)

['B00NHUW1NE', 'B005P1YV90', 'B00DOEA73O', 'B00KQ2VQDC', 'B00NHUVF18', 'B00D8HXHYI', 'B00DOE9QFY', 'B00M15DG80', 'B00NHUV8SI', 'B00NHUVTPA', 'B00NHUVFGS', 'B00NHUVN10', 'B00NHUW1UW', 'B00NHUVTSM', 'B00M32FLL6', 'B00KQ57KWK', 'B00LNN1EZ8', 'B00R9LA030', 'B00D8J693W', 'B00IF3BG6G']


In [40]:
# create a dataframe for the unpaid group and filter the rows for only the products rated in the paid group
# and calculate the average rating for each product
not_vine_products_df = filtered_products_df.filter(col("vine") == 'N').filter(col("product_id").isin(vine_products)).groupBy("product_id").agg(avg("star_rating")).withColumnRenamed("avg(star_rating)", "non-Vine avg rating")
not_vine_products_df.show()

+----------+-------------------+
|product_id|non-Vine avg rating|
+----------+-------------------+
|B00NHUW1NE|                5.0|
|B00D8HXHYI|                4.0|
|B00NHUVFGS|                1.0|
|B00NHUW1UW|                5.0|
|B00NHUVTSM|                5.0|
+----------+-------------------+



In [42]:
# compare the average rating for each product in both paid and unpaid group
avg_rating_paid_vs_unpaid = vine_products_df.join(not_vine_products_df, on="product_id", how="inner")
avg_rating_paid_vs_unpaid.show()

+----------+---------------+-------------------+
|product_id|Vine avg rating|non-Vine avg rating|
+----------+---------------+-------------------+
|B00NHUW1NE|            5.0|                5.0|
|B00D8HXHYI|            5.0|                4.0|
|B00NHUVFGS|            4.5|                1.0|
|B00NHUW1UW|            4.0|                5.0|
|B00NHUVTSM|            5.0|                5.0|
+----------+---------------+-------------------+

